In [2]:
cd ..

/home/dmitriishubin/Desktop/physionet-challenge-2020


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import os
import gc
from tqdm import tqdm
import matplotlib.pyplot as plt
from metrics.metrics import Metric
from kardioml.data.resample import Resampling

from kardioml.data.p_t_wave_detector import PTWaveDetection

metric = Metric()
resampling = Resampling()

In [4]:
%matplotlib qt

In [5]:
#DATASETS = ['A','B','C','D','E','F']
DATASETS = ['A','B','D','E'] #current for training
#DATASETS = ['D']
datalist = []

for dataset in DATASETS:
    files = [i[:-4] for i in os.listdir(f'./data/{dataset}/formatted/') if i.find('.npy')!=-1]
    for file in files:
        datalist.append(f'./data/{dataset}/formatted/'+file)



# Check the length fits json and numpy

In [ ]:
for data in tqdm(datalist):
    signal = np.load(data+'.npy')
    meta = json.load(open(data+'.json'))
    
    if meta['shape'][0] != signal.shape[0]:
        break
    

# Check the length distribution, all datasets

In [ ]:
length_list = []
exclusions = []
exclusions_labels = []
exclusions_digits = []

for data in tqdm(datalist):
    meta = json.load(open(data+'.json'))
    if meta['labels_training_merged'] is None:
        continue
    if meta['shape'][0] > 38000:
        file_name = data.split('/')     
        exclusions.append(file_name[-1])
        exclusions_labels.append(meta['labels_full'])
        exclusions_digits.append(meta['labels_training_merged'])
    length_list.append(meta['shape'][0])

    
sns.distplot(length_list)

In [ ]:
hist = np.histogram(length_list,2000)
plt.plot(hist[1][:-1][:100],hist[0])

In [ ]:
np.percentile(length_list,99)

# Check distribution of classes, calculate weights

In [10]:
labels = []

for data in tqdm(datalist):
    meta = json.load(open(data+'.json'))
    if meta['labels_training_merged'] is None:
        meta['labels_training_merged'] = [0]*27
    
    label = meta['labels_training_merged']
    if label[4] > 0 or label[18] >0:
        label[4] = 1
        label[18] = 1
    if label[23] > 0 or label[12] >0:
        label[23] = 1
        label[12] = 1
    if label[26] > 0 or label[13] >0:
        label[26] = 1
        label[13] = 1
    labels.append(label)
    

    
labels = np.array(labels)

100%|██████████| 32683/32683 [02:37<00:00, 207.99it/s]


In [14]:
weights_arr = np.array([1.]*27)
weights_arr[[4,18,23,12,13,26]] = 0.5
weights_arr

array([1. , 1. , 1. , 1. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.5,
       0.5, 1. , 1. , 1. , 1. , 0.5, 1. , 1. , 1. , 1. , 0.5, 1. , 1. ,
       0.5])

In [40]:
weights = np.sum(labels,axis=0)
weights = weights/labels.shape[0]
s = (np.mean(weights[:21]) + np.mean(weights[22:]))/2
weights[:21] = s
weights[22:] = s
weights = weights[21]/weights
weights = weights / np.max(weights)
weights = weights * weights_arr
weights = weights.tolist()
weights

[1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 0.5,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 1.0,
 1.0,
 0.05800750173327947,
 1.0,
 0.5,
 1.0,
 1.0,
 0.5]

In [37]:
s

0.033889032160304686

In [39]:
sns.distplot(weights)

<AxesSubplot:>

# Checking PT detector perfomance

In [ ]:
data_range = np.arange(len(datalist))[0:11].tolist()

datalist_sub = [i for index,i in enumerate(datalist) if index in data_range]


for data in tqdm(datalist_sub):
    meta = json.load(open(data+'.json'))
    if meta['shape'][0] > 38000:
        continue
    elif meta['labels_training_merged'] is None:
        continue
    else:
        signal = np.load(data+'.npy')
        
        
        fig = plt.figure(figsize=(20,10))
        for i in range(12):
            channel = signal[:,i]
            plt.plot(channel+i*500)
            plt.plot(meta['p_waves'][i],channel[meta['p_waves'][i]]+i*500,'*')
            plt.plot(meta['t_waves'][i],channel[meta['t_waves'][i]]+i*500,'*')
        plt.show()
        plt.title(str(meta['labels_full'])+' | '+data)
        print('===================')
        print(data)
        print(meta['labels_full'])
        print('===================')
        if data == '../data/A/formatted/A2091':
            break

# Checking downsampling algo

In [207]:
data_range = np.arange(len(datalist))[0:11].tolist()

datalist_sub = [i for index,i in enumerate(datalist) if index in data_range]


for data in tqdm(datalist_sub):
    meta = json.load(open(data+'.json'))
    if meta['shape'][0] > 38000:
        continue
    elif meta['labels_training_merged'] is None:
        continue
    else:
        signal = np.load(data+'.npy')
        
        
        fig = plt.figure(figsize=(20,10))
        for i in range(12):
            channel = signal[:,i]
            plt.plot(channel+i*500)
            channel = resampling.downsample(channel,order=2)
            plt.plot(channel+i*500)
        plt.show()
        plt.title(str(meta['labels_full'])+' | '+data)
        print('===================')
        print(data)
        print(meta['labels_full'])
        print('===================')
        if data == '../data/A/formatted/A2091':
            break

 36%|███▋      | 4/11 [00:00<00:00, 16.97it/s]

./data/A/formatted/A3306
['right bundle branch block']
./data/A/formatted/A6853
['sinus rhythm']
./data/A/formatted/A3233
['1st degree av block']
./data/A/formatted/A3855
['atrial fibrillation']


 82%|████████▏ | 9/11 [00:00<00:00, 12.37it/s]

./data/A/formatted/A5826
['1st degree av block']
./data/A/formatted/A6747
['right bundle branch block']
./data/A/formatted/A0178
['right bundle branch block']


100%|██████████| 11/11 [00:00<00:00, 12.10it/s]

./data/A/formatted/A5676
['atrial fibrillation']
./data/A/formatted/A3929
['1st degree av block']


# CHecking median filtering

In [7]:
from scipy.signal import medfilt

data_range = np.arange(len(datalist))[0:11].tolist()

datalist_sub = [i for index,i in enumerate(datalist) if index in data_range]


for data in tqdm(datalist_sub):
    meta = json.load(open(data+'.json'))
    if meta['shape'][0] > 38000:
        continue
    elif meta['labels_training_merged'] is None:
        continue
    else:
        signal = np.load(data+'.npy')
        
        
        
        fig = plt.figure(figsize=(20,10))
        for i in range(12):
            channel = signal[:,i]
            channel -= medfilt(channel,501)
            plt.plot(channel+i*500)
        plt.show()
        plt.title(str(meta['labels_full'])+' | '+data)
        print('===================')
        print(data)
        print(meta['labels_full'])
        print('===================')
        if data == '../data/A/formatted/A2091':
            break

  9%|▉         | 1/11 [00:04<00:41,  4.16s/it]

./data/A/formatted/A3306
['right bundle branch block']


 27%|██▋       | 3/11 [00:06<00:25,  3.20s/it]

./data/A/formatted/A6853
['sinus rhythm']


 36%|███▋      | 4/11 [00:08<00:20,  2.89s/it]

...

./data/A/formatted/A0178
['right bundle branch block']


 91%|█████████ | 10/11 [00:22<00:02,  2.50s/it]

./data/A/formatted/A5676
['atrial fibrillation']


100%|██████████| 11/11 [00:24<00:00,  2.23s/it]

./data/A/formatted/A3929
['1st degree av block']


# Get list of records without labels

In [12]:
#DATASETS = ['A','B','C','D','E','F']
DATASETS = ['A','B','D','E'] #current for training
#DATASETS = ['D']
datalist_nones = []

for dataset in DATASETS:
    files = [i[:-4] for i in os.listdir(f'./data/{dataset}/formatted/') if i.find('.npy')!=-1]
    for file in tqdm(files):
        meta = json.load(open(f'./data/{dataset}/formatted/'+file+'.json'))
        if meta['labels_training_merged'] is None:
            datalist_nones.append(meta['filename'])

100%|██████████| 21837/21837 [01:19<00:00, 276.02it/s]


In [17]:
dataset_json = {}
dataset_json['data'] = datalist_nones

json.dump(dataset_json, open("additional_data.json","w"))